In [13]:
import re
import numpy as np
import pandas as pd
import unicodecsv as csv
import itertools

# Analysize variables.

In [3]:
# Read documentation.
columns_file = r"../records_documentation.txt"

with open(columns_file,"r") as f:
    columns = f.readlines()
columns

['[Column 0] ID\n',
 '[Column 1] Birth year: deduced from education history, where available\n',
 '[Column 2] Gender flag: 1=female, 2=male, 0=unknown\n',
 '[Column 3] Skillset1: primary skillset deduced from self-reported skills\n',
 '[Column 4] Skillset1 weight: the extent to which primary skillset is representative of overall self-reported skills\n',
 '[Column 5] Skillset2: secondary skillset deduced from self-reported skills\n',
 '[Column 6] Skillset2 weight: the extent to which secondary skillset is representative of overall self-reported skills\n',
 '[Column 7] City of profile (does not change): where unavailable, the field is left blank\n',
 '[Column 8] Country of Profile (does not change)\n',
 '[Column 9] Education (highest degree attained): 0=none/unknown; 1=high school; 2=vocational degree; 3=associateÕs; 4=bachelorÕs; 5=masterÕs other than MBA; 6=MBA; 7=doctorate (PhD/JD/MD) [NB: Romanian names not covered]\n',
 "[Column 10] Elite institution: a flag indicating whether any o

So:
- Unique identifier: ID.
- Variable Groups:
    - Column 0-10, profile info; 
    - Column 11-28, employment or education info; (c26 = False or True)
        - Column 11-18, general info;
        - Column 19-25, company info;
        - Column 27-28, school info.

In [4]:
# Get unique values for each variable.
empl_file = r"../deloitte_pwc.csv"

unique_values = []
for i in range(33):
    unique_values.append(set())
with open(empl_file,"rb") as f:
    reader = csv.reader(f,encoding='utf-8',escapechar='',delimiter='\t')
    for idx, line in enumerate(reader):
        for i in range(33):
            unique_values[i].add(line[i])
for idx, values in enumerate(unique_values):
    print("[Column {}] ".format(idx)+str(len(values)))
        

[Column 0] 1095185
[Column 1] 84
[Column 2] 3
[Column 3] 45
[Column 4] 390501
[Column 5] 45
[Column 6] 390503
[Column 7] 16127
[Column 8] 258
[Column 9] 8
[Column 10] 2
[Column 11] 849
[Column 12] 3
[Column 13] 763
[Column 14] 3
[Column 15] 2
[Column 16] 3246
[Column 17] 2640332
[Column 18] 865
[Column 19] 1767657
[Column 20] 1557541
[Column 21] 10261
[Column 22] 309
[Column 23] 2
[Column 24] 204454
[Column 25] 739
[Column 26] 2
[Column 27] 8
[Column 28] 2
[Column 29] 1469
[Column 30] 68
[Column 31] 7835
[Column 32] 1003970


In [45]:
# One block to convert data to pd.DataFrame.
empl_file = r"../deloitte_pwc.csv"

entries_lst = []
with open(empl_file,"rb") as f:
    reader = csv.reader(f,encoding='utf-8',escapechar='',delimiter='\t')
    for idx, line in enumerate(reader):
        entries_lst.append(line)
        if idx == 100:
            break

samples_lst = entries_lst
entries_df = pd.DataFrame(data=entries_lst)
entries_df



,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,ed3c50a8-d2d3-3809-9d2a-0f53574560aa,1980,2,Accounting and Auditing,0.9292892909340925,Industrial Management,0.00825650305491638,Paris,France,5,...,False,,,True,5,False,,,,1525765406000
1,ed3c50a8-d2d3-3809-9d2a-0f53574560aa,1980,2,Accounting and Auditing,0.9292892909340925,Industrial Management,0.00825650305491638,Paris,France,5,...,False,,,True,,False,,,,1525765406000
2,ed3c50a8-d2d3-3809-9d2a-0f53574560aa,1980,2,Accounting and Auditing,0.9292892909340925,Industrial Management,0.00825650305491638,Paris,France,5,...,False,,,False,,False,,,,1525765406000
3,ed3c50a8-d2d3-3809-9d2a-0f53574560aa,1980,2,Accounting and Auditing,0.9292892909340925,Industrial Management,0.00825650305491638,Paris,France,5,...,False,,54,False,,False,,,,1525765406000
4,ed3c50a8-d2d3-3809-9d2a-0f53574560aa,1980,2,Accounting and Auditing,0.9292892909340925,Industrial Management,0.00825650305491638,Paris,France,5,...,False,"paris area, france",54,False,,False,,,,1525765406000
5,ed3c50a8-d2d3-3809-9d2a-0f53574560aa,1980,2,Accounting and Auditing,0.9292892909340925,Industrial Management,0.00825650305491638,Paris,France,5,...,False,"paris area, france",48,False,,False,,,,1525765406000
6,ed3c50a8-d2d3-3809-9d2a-0f53574560aa,1980,2,Accounting and Auditing,0.9292892909340925,Industrial Management,0.00825650305491638,Paris,France,5,...,False,"paris area, france",92,False,,False,,,,1525765406000
7,c8f59145-e5e3-39ba-aadc-e076e6390b28,2000,2,-1,0,-1,0,Hasselt,Belgium,0,...,False,,561,False,,False,,,,1527648032000
8,c8f59145-e5e3-39ba-aadc-e076e6390b28,2000,2,-1,0,-1,0,Hasselt,Belgium,0,...,False,,54,False,,False,,,,1527648032000
9,c8f59145-e5e3-39ba-aadc-e076e6390b28,2000,2,-1,0,-1,0,Hasselt,Belgium,0,...,False,"hyderabad area, india",561,False,,False,,,,1527648032000


In [46]:
# Get an example.
entry = samples_lst[5]
for i in range(len(entry)):
    print("[Column {}] ".format(i)+entry[i])

[Column 0] ed3c50a8-d2d3-3809-9d2a-0f53574560aa
[Column 1] 1980
[Column 2] 2
[Column 3] Accounting and Auditing
[Column 4] 0.9292892909340925
[Column 5] Industrial Management
[Column 6] 0.00825650305491638
[Column 7] Paris
[Column 8] France
[Column 9] 5
[Column 10] False
[Column 11] 2012-10-01
[Column 12] True
[Column 13] 2016-07-01
[Column 14] True
[Column 15] False
[Column 16] 1369
[Column 17] Internal Auditor,internal auditor
[Column 18] 
[Column 19] SYSTRA
[Column 20] systra
[Column 21] 
[Column 22] FRANCE
[Column 23] False
[Column 24] paris area, france
[Column 25] 48
[Column 26] False
[Column 27] 
[Column 28] False
[Column 29] 
[Column 30] 
[Column 31] 
[Column 32] 1525765406000


# Aggregate hiring/firing to the firm-by-date level.

In [66]:
# Read, Identify & Aggregate.
empl_file = r"../deloitte_pwc.csv"

## Initialize.
career_lst = [] # Will contain the entries for the same person.
person_this = None
profile_this = None

## Iterate.
with open(empl_file,"rb") as f:
    reader = csv.reader(f,encoding='utf-8',escapechar='',delimiter='\t')
    for idx, entry in enumerate(itertools.chain(reader,[None]*33)):
        if idx == 1000:
            break
        if person_this != entry[0]:
            if career_lst != []:
                ### Analyze.
                career_df = pd.DataFrame(career_lst)
                print(career_df[[0,1,2,3,4,10]])
                print(profile_this)
            ### Re-initialize.
            person_this = entry[0]
            profile_this = entry[1:11]
            career_lst = []
        ### Record this entry.
        if profile_this[2] == 'Accounting and Auditing' and entry[26] == 'False': # only consider accountants.
            career_lst.append(entry[11:29])
            


           0      1           2      3      4  10
0  2004-09-01   True  2005-06-01   True  False   
1  2006-07-01   True  2006-11-01   True  False  P
2  2007-01-01   True  2012-10-01   True  False  P
3  2012-10-01   True  2016-07-01   True  False   
4  2017-01-01  False        None  False   True   
['1980', '2', 'Accounting and Auditing', '0.9292892909340925', 'Industrial Management', '0.00825650305491638', 'Paris', 'France', '5', 'False']
           0      1           2      3      4         10
0  2001-01-01  False  2001-12-01  False  False  TIME_OFF
1  2002-09-01   True  2003-02-01   True  False         D
2  2003-02-01   True  2004-07-01   True  False         D
3  2004-09-01   True  2005-02-01   True  False         D
4  2005-09-01   True  2008-09-01   True  False         D
5  2008-08-01   True  2013-07-01   True  False         D
6  2013-07-01   True  2016-11-01   True  False          
7  2016-11-01   True  2017-04-01   True  False          
8  2017-04-01   True        None  False   T